In [1]:
import numpy as np
import pandas as pd
import sys

loc = "../../tools/SERGIO/"
sys.path.append(loc+"SERGIO")
from sergio import sergio

In [2]:
sim = sergio(number_genes=1200, number_bins = 2, number_sc = 500, noise_params = 1, decays=0.8, sampling_state=15, noise_type='dpd')
sim.build_graph(input_file_taregts =loc+'data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Interaction_cID_6.txt', input_file_regs=loc+'data_sets/De-noised_1200G_9T_300cPerT_6_DS3/Regs_cID_6.txt', shared_coop_state=2)

In [49]:
import decoupler as dc

net = dc.get_dorothea(organism="human")

In [50]:
net[net.confidence != 'C'].target.value_counts()

target
MYC       48
CDKN1A    42
CCND1     40
BCL2      32
RUNX1     31
          ..
AMZ1       1
GDAP1      1
A4GALT     1
GEN1       1
GGH        1
Name: count, Length: 5150, dtype: int64

In [37]:
s = net.source.value_counts()[(net.source.value_counts()>5) & (net.source.value_counts()<200)].index.tolist()
len(s)

263

In [42]:
t = net[net.source.isin(s)].target.value_counts()[net[net.source.isin(s)].target.value_counts()>3].index.tolist()
net = net[net.source.isin(s) & (net.target.isin(t))]

In [45]:
net.weight.value_counts()

weight
 0.333333    6218
 1.000000     952
 0.500000     399
-1.000000     154
Name: count, dtype: int64

In [ ]:
nnet = []
net.sort_values(by="weight", ascending=False)
for i in set(net.target):
    for n, j in enumerate(net.loc[net.target == i].iterrows()):
        toa = [i, ""]
        if j.weight > 0:
            toa[1] += " or " + j.source
                nnet.append([j.source, j.target])
        else:
            toa[1] += " or not(" + j.source + ")"
        
net = pd.DataFrame(nnet, columns = ['source', 'target', 'weight'])
        

In [43]:
# make it ready for boolODE
for i in net.iterrows():
    if i.weight < 0:
        net.at[i.Index, 'target'] = "not("+i.Rule+")"
net = net.drop(columns = ['confidence', 'weight']).rename(columns = {'source':'Gene', 'target':'Rule'})


,source,confidence,target,weight
13,FLI1,A,ID2,1.000000
15,ETV4,A,TNF,1.000000
16,ETV4,A,TIMP2,1.000000
17,ETV4,A,BDKRB1,1.000000
19,ETS2,A,VWF,1.000000
...,...,...,...,...
32264,LEF1,C,PRUNE2,0.333333
32266,LEF1,C,PRKCH,0.333333
32269,LEF1,C,NLGN1,0.333333
32270,LEF1,C,NKD1,0.333333


In [1]:
from omnipath import regnetwork_download

ImportError: cannot import name 'regnetwork_download' from 'omnipath' (/home/ml4ig1/Documents code/scPRINT/.venv/lib/python3.10/site-packages/omnipath/__init__.py)

In [ ]:
regnetwork_download(organism = "human")

In [3]:
sim.simulate()

expr = sim.getExpressions()
expr = np.concatenate(expr, axis = 1)

Start simulating new level
There are 2 genes to simulate in this layer
Done with current level
Start simulating new level
There are 3 genes to simulate in this layer
Done with current level
Start simulating new level
There are 5 genes to simulate in this layer
Done with current level
Start simulating new level
There are 9 genes to simulate in this layer
Done with current level
Start simulating new level
There are 26 genes to simulate in this layer
Done with current level
Start simulating new level
There are 82 genes to simulate in this layer
Done with current level
Start simulating new level
There are 1073 genes to simulate in this layer
Done with current level


In [9]:
np.save("expr.npy", expr)

In [76]:
expr = np.load("expr.npy")

In [77]:
expr = sim.outlier_effect(expr, outlier_prob=0.05, mean=10, scale=1)
se, expr = sim.lib_size_effect(expr, mean=8, scale=0.3)
binary_ind = sim.dropout_indicator(expr, shape=1000, percentile=0.9)
expr = np.multiply(binary_ind, expr)
expr = sim.convert_to_UMIcounts(expr)
expr = np.concatenate(expr, axis = 1).T

In [83]:
val = pd.read_csv(loc+'data_sets/De-noised_1200G_9T_300cPerT_6_DS3/gt_GRN.csv')
val

,1,2
0,1,0
1,4,7
2,4,15
3,4,13
4,4,16
...,...,...
2707,1177,1190
2708,1177,842
2709,1177,1195
2710,1177,552


In [95]:
val = val.astype(str)

In [96]:
from grnndata import from_adata_and_longform
from anndata import AnnData

grn = from_adata_and_longform(AnnData(X=expr), val, has_weight=False)

  0%|          | 0/2712 [00:00<?, ?it/s]

100%|██████████| 2712/2712 [00:00<00:00, 19992.01it/s]


In [97]:
grn

GRnnData object with n_obs × n_vars = 9000 × 1200
    varp: 'GRN'
    with a grn of 2712 elements

In [99]:
grn.write_h5ad("../../data/SERGIO_GRN.h5ad")